In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB

import pickle

from all_model_py import CutDebt, IDClassifier, IfKnowDebtor, Installment, WillingToPay, ConfirmLoan

# import sys
# sys.path.append('../CutDebt/')
# sys.path.append('../IDClassifier/')
# sys.path.append('../IfKnowDebtor/')
# sys.path.append('../Installment/')
# sys.path.append('../SetDueDay/')
# sys.path.append('../WillingToPay/')

# from CutDebt_py import CutDebt
# from IDClassifier_py import IDClassifier
# from IfKnowDebtor_py import IfKnowDebtor
# from Installment_py import Installment
# from SetDueDay_py import SetDueDay
# from WillingToPay_py import WillingToPay

In [3]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
func_list = [CutDebt,IDClassifier,IfKnowDebtor,Installment,ConfirmLoan,WillingToPay]

ind = 0
for each_model in model_list:
    print(each_model)
    
    # load data
    path = '../../data/{}/'
    data = pd.read_csv(path.format(each_model) + 'combined_mock_up_data.csv', encoding='utf8')
    
    # get tfidf
    phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

    print('fitting phrase')
    phrase_vectorizer.fit(data.split_text)

    print('transform phrase')
    phrase = phrase_vectorizer.transform(data.split_text)

    
    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(phrase, data.label)
    
    
    # logistic
    log_r = LogisticRegression()
    log_r.fit(phrase, data.label)
    
    
#     # lightGBM
#     le = preprocessing.LabelEncoder()
#     le.fit(data.label.values)
#     onelabels = le.transform(data.label.values)
#     multicoder = MultiLabelBinarizer()
#     lables = multicoder.fit_transform([data.label.values])
#     if each_model == 'WillingToPay':
#         params = {
#         'learning_rate': 0.2,
#         'num_iterations':1000,
#         'application': 'multiclassova',
#         'num_class': 4,
#         'num_leaves': 31,
#         'verbosity': -1,
#         'metric': 'multi_error',
#         'data_random_seed': 2,
#     #     'bagging_fraction': 0.8,
#     #     'feature_fraction': 0.6,
#         'nthread': 4,
#         'lambda_l1': 1,
#         'lambda_l2': 1,
#         'early_stopping_round':200
#         } 
#     else:        
#         params = {
#         'learning_rate': 0.2,
#         'num_iterations':1000,
#         'application': 'multiclassova',
#         'num_class': 3,
#         'num_leaves': 31,
#         'verbosity': -1,
#         'metric': 'multi_error',
#         'data_random_seed': 2,
#     #     'bagging_fraction': 0.8,
#     #     'feature_fraction': 0.6,
#         'nthread': 4,
#         'lambda_l1': 1,
#         'lambda_l2': 1,
#         'early_stopping_round':200
#         } 
#     # lgbm_train = lgb.Dataset(phrase, data.label)
#     lgbm_train = lgb.Dataset(phrase, onelabels)
#     lgbm_val = lgb.Dataset(phrase, onelabels)
#     lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)
    
    
    # Naive Bayes
    nb = MultinomialNB()
    nb.fit(phrase, data.label)
    
    
    
    
    
    
    print('finish training')
    
    # save model
    save_path = '../../savedModel/{}/'
    # save tfidf
    pickle.dump(phrase_vectorizer, open(save_path.format(each_model) + "tfidf.pickle", "wb"))
    # save linear svc
    pickle.dump(lsvc, open(save_path.format(each_model) + "LinearSVC.pickle", "wb"))
    # save logistic
    pickle.dump(log_r, open(save_path.format(each_model) + "Logistic.pickle", "wb"))
    # save lightGBM
#     pickle.dump(lgbm_model, open(save_path.format(each_model) + "Lgbm.pickle", "wb"))
    # save naive bayes
    pickle.dump(nb, open(save_path.format(each_model) + "nb.pickle", "wb"))

    
    
    
    func = func_list[ind]
    result = func(svc=lsvc, logistic=log_r, nb=nb, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')
    pickle.dump(result, open(save_path.format(each_model) + each_model + '.pickle', "wb"))
    ind = ind + 1
    
    
    
    

CutDebt
fitting phrase
transform phrase


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


finish training


Loading model cost 0.491 seconds.
Prefix dict has been built succesfully.


IDClassifier
fitting phrase


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [9]:
idc = pickle.load(open("../../savedModel/IDClassifier/IDClassifier.pickle", 'rb'))
cutd = pickle.load(open("../../savedModel/CutDebt/CutDebt.pickle", 'rb'))
ifk = pickle.load(open("../../savedModel/IfKnowDebtor/IfKnowDebtor.pickle", 'rb'))
will = pickle.load(open("../../savedModel/WillingToPay/WillingToPay.pickle", 'rb'))
inst = pickle.load(open("../../savedModel/Installment/Installment.pickle", 'rb'))
setd = pickle.load(open("../../savedModel/SetDueDay/SetDueDay.pickle", 'rb'))

In [24]:
idc.classify('我不是')

(1, [1, 0.99945433087829494])

In [29]:
import jieba

seg_list = jieba.cut('我也没有啊', cut_all=False)
" ".join(seg_list)

'我 也 没有 啊'

In [7]:
a = np.random.rand(3,2)
print(a)
np.mean(a,axis = 0)

[[0.02349722 0.81760363]
 [0.30495461 0.08976253]
 [0.65089555 0.16133172]]


array([0.32644913, 0.35623263])